In [4]:
import numpy as np
import pandas as pd 
import json
import spacy

from src.nlp.patterns.day_in_my_life import DAY_IN_MY_LIFE_PATTERNS
from src.nlp.patterns.dupe import DUPE_PATTERNS
from src.nlp.patterns.get_ready_with_me import GRWM_PATTERNS
from src.nlp.patterns.point_of_view import POINT_OF_VIEW_PATTERNS
from src.nlp.patterns.restocking import RESTOCKING_PATTERNS
from src.nlp.patterns.unboxing import UNBOXING_PATTERNS
from src.nlp.text_utils.text_contains_word import detect_patterns, ellipsis
from src.nlp.patterns.call_to_action import CTA_PATTERNS
from src.nlp.patterns.hack import HACK_PATTERNS
from src.nlp.patterns.behind_the_scenes import BTS_PATTERNS
from src.nlp.patterns.brand_matcher import ALL_BRAND_MATCHING_PHRASES
from src.nlp.patterns.everyday_taks import TIPS_PATTERNS
from src.nlp.text_utils.text_counter import count_two_words, paid_partnership, findall_counter
from src.nlp.text_utils.text_emoji import text_has_emoji
from src.nlp.patterns.brand_matcher import BRAND_PATTERNS
from src.nlp.patterns.hook import HOOK_PATTERNS
from src.nlp.patterns.direct_questions import DIRECT_QUESTION_PATTERNS
from src.nlp.patterns.integration import INTEGRATION_PATTERNS, IntegrationPattern
from src.nlp.patterns.curiosity import CURIOSITY_PATTERNS

from src.nlp.text_utils.similarity import compare_first_and_last_sentence
from src.cv.video.helpers import scene_num, get_video_duration
from src.audio.analyze import energetic_music
from src.nlp.text_utils.general_utils import second_person
from src.ts.parser import extract_seconds_from_timestamps
from scenedetect import detect, AdaptiveDetector

from app.xgboost_service import XGBoostService
from app.xgboost_adapter import XGBoostAdapter

nlp = spacy.load("en_core_web_sm")

In [5]:
##### Load the data

with open("output/transcription/wav_file_text_mapping.json", "rb") as f: 
    audio_text = json.load(f)

with open("output/dataset_info/influencer_data_set_info.json", "rb") as f: 
    influencer_data = json.load(f)

##### Load raw audios and get their keys
with open("output/transcription/transcription_results.json", "rb") as f: 
    raw_audios = json.load(f)

audio_keys = list(raw_audios.keys())

In [6]:
df = pd.DataFrame.from_records(audio_text, index=[0]).T.reset_index()
df.columns = ['path', 'transcription']

In [ ]:
RANDOM_SEED = 42
HOOK_THRESHOLD = 6
KEYWORD_SPONSOR = "sponsor"

1: Does the introduction of the brand align with the wider video narrative?

In [7]:
df

,path,transcription
0,results/coterie/C-5_QPIyNVK_2024-08-20_21-17-5...,I just woke my baby up from a four hour nap. ...
1,results/coterie/C-DXZgbOkpL_2024-07-30_15-54-2...,Thank you.
2,results/coterie/C-ENTmNu4LL_2024-07-30_23-47-3...,Thank you.
3,results/coterie/C-Nbj5Ss_kr_2024-08-03_13-43-1...,Thank you. Thank you.
4,results/coterie/C-V6tW3u8fR_2024-08-06_20-54-4...,Hey! Hey! Hey! Hey! Hey!
...,...,...
387,results/theninjamas/DBZadHsPgLS_2024-10-21_19-...,See how I do my night time routine? Once I pi...
388,results/theninjamas/DBZcfWGSbJo_2024-10-21_19-...,"Nighttime routines can be chaotic, but thanks..."
389,results/theninjamas/DBZyJxJSi7B_2024-10-21_22-...,Between home renovations during the day and n...
390,results/theninjamas/DBaWN8rp6tg_2024-10-22_03-...,These are our top potty training tips partner...


2: Brand introduced as the sponsor of the video

In [9]:
brand_patterns = list(ALL_BRAND_MATCHING_PHRASES)

transcription_ts = [[(raw_audios[audio]["segments"][ts]["text"], raw_audios[audio]["segments"][ts]["start"], raw_audios[audio]["segments"][ts]["end"]) for ts in range(len(raw_audios[audio]["segments"]))] for audio in audio_keys]

# We get tuples with the start and end ts where brand is matched
ts_brand = [[(transcription_ts[i][j][1], transcription_ts[i][j][2]) if findall_counter(transcription_ts[i][j][0], brand_patterns) else 0 for j in range(len(transcription_ts[i]))] for i in range(len(transcription_ts))]

# Get the first tupple where the values are not 0
first_brand_appearance = [next((x for x in lst_values if x), 0) for lst_values in ts_brand]

# Get the average (orientative value where the name of the brand is verbalized)
avg_ts_brand_appearance = [round(float(np.mean(values)), 2) for values in first_brand_appearance]

# Get the dataframe
brand_ts_df = pd.DataFrame({"path": audio_keys, "BRAND_TS": avg_ts_brand_appearance})

# Get the path
brand_ts_df["path"] = brand_ts_df["path"].apply(lambda x: x.split(".wav")[0])


# Brand as sponsor in integration phase
first_brand_appearance_start = [next((x[0] for x in lst_values if x), 0) for lst_values in ts_brand]

texts_with_brand_start = [[transcription_ts[i][j] if (transcription_ts[i][j][1] == first_brand_appearance_start[i]) and (first_brand_appearance_start[i] != 0) else "" for j in range(len(transcription_ts[i]))] for i in range(len(transcription_ts))]
texts_with_brand_start = [[tpl for tpl in segments if tpl != ''] for segments in texts_with_brand_start]

NameError: name 'ALL_BRAND_MATCHING_PHRASES' is not defined

3-4: Background analysis -> Integration filmed in the same setting as the wider video and previous scene

5 - 6 / 163 - 164: Duration of ad

In [ ]:
### Get Video and Image paths


video_paths = [video for sublist in influencer_data.values() for video in sublist["videos"]]
image_paths = [video for sublist in influencer_data.values() for video in sublist["images"]]

##### Video duration
video_durations = [get_video_duration(video) for video in video_paths]

video_df = pd.DataFrame({"path":video_paths, "video_duration": video_durations})



# 163 and 164
video_df["less_than_30_seconds"] = video_df["video_durations"].apply(lambda x: 1 if x <= 30 else 0)


# Normalize path

df["path"] = df["path"].apply(lambda x: x.split(".wav")[0])
video_df["path"] = video_df["path"].apply(lambda x: x.split(".mp4")[0])

df = df.merge(video_df, on="path", how="left")

df["video_duration"] = df["video_duration"].fillna(0)

# df.to_csv("saved_data/initial_df_1.csv", index=False)

In [10]:
# Ad length less than 6 seconds
df["less_than_6_seconds"] = df["video_duration"].apply(lambda x: 1 if x <= 6 else 0)

In [ ]:
# Ad length between 15 and 20 seconds
df["between_15_and_20_seconds"] = df["video_duration"].apply(lambda x: 1 if x > 15 and x < 20 else 0)

7: Audience in central position

8: Brand logo shown

CTA: 9-15 (missing visual)

In [11]:
# transcription_df
transcription_df = detect_patterns(df = df, patterns = CTA_PATTERNS, column="transcription")
transcription_df.rename(
    columns=lambda col: col + "_transcription" if col.startswith("CTA") else col,
    inplace=True
)
# Visual

# ocr_df.rename(
#     columns=lambda col: col + "_ocr" if col.startswith("CTA") else col,
#     inplace=True
# )

In [12]:
# On visual
# cta_columns = [col for col in ocr_df.columns if "cta" in col]
# ocr_df["CTA_STICKER"] = ocr_df[cta_columns].any(axis=1)

,path,transcription,CTA_BUY,CTA_GIFT,CTA_READ,CTA_SIGN_UP,CTA_FIND_OUT,CTA_LEARN,CTA_SUBSCRIBE,CTA_SHARE,CTA_DOWNLOAD,CTA_VISIT,CTA_CONTACT,CTA_PARTICIPATE
0,results/coterie/C-5_QPIyNVK_2024-08-20_21-17-5...,I just woke my baby up from a four hour nap. ...,False,False,False,False,False,False,False,False,False,False,False,False
1,results/coterie/C-DXZgbOkpL_2024-07-30_15-54-2...,Thank you.,False,False,False,False,False,False,False,False,False,False,False,False
2,results/coterie/C-ENTmNu4LL_2024-07-30_23-47-3...,Thank you.,False,False,False,False,False,False,False,False,False,False,False,False
3,results/coterie/C-Nbj5Ss_kr_2024-08-03_13-43-1...,Thank you. Thank you.,False,False,False,False,False,False,False,False,False,False,False,False
4,results/coterie/C-V6tW3u8fR_2024-08-06_20-54-4...,Hey! Hey! Hey! Hey! Hey!,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,results/theninjamas/DBZadHsPgLS_2024-10-21_19-...,See how I do my night time routine? Once I pi...,False,False,False,False,False,False,False,False,False,False,False,False
388,results/theninjamas/DBZcfWGSbJo_2024-10-21_19-...,"Nighttime routines can be chaotic, but thanks...",False,False,False,False,False,False,False,False,False,False,False,False
389,results/theninjamas/DBZyJxJSi7B_2024-10-21_22-...,Between home renovations during the day and n...,False,False,False,False,False,False,False,False,False,False,False,False
390,results/theninjamas/DBaWN8rp6tg_2024-10-22_03-...,These are our top potty training tips partner...,False,False,False,False,False,False,False,False,False,False,False,False


16 Single call to action

In [ ]:
df = ocr_df.merge(transcription_df, how="left", on="path")

cta_columns = [col for col in df.columns if "CTA" in col and col != "CTA_STICKER"]
df["CTA_SINGLE"] = df[cta_columns].sum(axis=1) == 1

17 / 80: First and last sentence

In [55]:
# # transcription_ts = [[(raw_audios[audio]["segments"][ts]["text"], raw_audios[audio]["segments"][ts]["start"], raw_audios[audio]["segments"][ts]["end"]) for ts in range(len(raw_audios[audio]["segments"]))] for audio in audio_keys]

# first_sentence = [[raw_audios[audio_key]["segments"][0]["text"]] for audio_key in audio_keys]
# last_sentence = [[raw_audios[audio_key]["segments"][-1]["text"]] for audio_key in audio_keys]

# first_sentence_set = [
#     set(' '.join(text).replace('.', '').replace(',', '').replace("'", "").lower().split())
#     for text in first_sentence
# ]

# last_sentence_set = [
#     set(' '.join(text).replace('.', '').replace(',', '').replace("'", "").lower().split())
#     for text in last_sentence
# ]


# def text_similarity(set_1, set_2): # Subject to change
#     return 1 if set_1 == set_2 else 0

# starts_ends_same_sentence = [text_similarity(first_sentence_set[i], last_sentence_set[i]) for i in range(len(first_sentence_set))]


# Apply this to the text
SIMILARITY_THRESHOLD = 0.5

df["same_sentence"] = df["transcription"].apply(lambda x: compare_first_and_last_sentence(x, SIMILARITY_THRESHOLD))

18 - 42 / 47 - 57 / 59 - 61 / 66 - 68: Content Type

In [22]:
# Transcription

df = detect_patterns(df = df, patterns = POINT_OF_VIEW_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = UNBOXING_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = GRWM_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = RESTOCKING_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = DAY_IN_MY_LIFE_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = DUPE_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = HACK_PATTERNS, column="transcription")
df = detect_patterns(df = df, patterns = BTS_PATTERNS, column="transcription") # Also 59-61
df = detect_patterns(df = df, patterns = TIPS_PATTERNS, column="transcription")

In [ ]:
# ocr

df = detect_patterns(df = df, patterns = POINT_OF_VIEW_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = UNBOXING_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = GRWM_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = RESTOCKING_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = DAY_IN_MY_LIFE_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = DUPE_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = HACK_PATTERNS, column="ocr")
df = detect_patterns(df = df, patterns = BTS_PATTERNS, column="ocr") # Also 59-61
df = detect_patterns(df = df, patterns = TIPS_PATTERNS, column="ocr")

43 - 46 / 62 - 65: Surprise elements

58: Close-up

69: Ad or #Ad

In [ ]:
df[['hashtag_ad_count', 'standalone_ad_count']] = pd.DataFrame(
    df['text'].apply(lambda x: count_two_words(x, "ad", "#ad")).tolist(), index=df.index
)

df['most_common'] = df.apply(
    lambda row: 'hashtag_ad' if row['hashtag_ad_count'] > row['standalone_ad_count'] else 
                'standalone_ad' if row['standalone_ad_count'] > row['hashtag_ad_count'] else
                'equal',
    axis=1
)

df["paid_partnership"] = df.apply(paid_partnership, axis=1)

70: Broad range of brand elements used

71 - 74: Emojis

In [ ]:
df["has_emoji"] = df["text"].apply(text_has_emoji)

75: Format align with the top performing format

76 - 79: Hook 

132: Negative hook: Loss aversion

In [ ]:
# General and negative hook

### This may need to be reworked since some of the 1st segments are much longer than 3 seconds


end_first_segment = [raw_audios[audio_keys[i]]["segments"][0]["end"] for i in range(len(audio_keys))]
text_first_segment = [raw_audios[audio_keys[i]]["segments"][0]["text"] for i in range(len(audio_keys))]

##### Hook first 3 seconds

hook_df = pd.DataFrame({"path": audio_keys, "text": text_first_segment, "end_first_segment": end_first_segment})

tmp_df = hook_df[hook_df["end_first_segment"] < HOOK_THRESHOLD]
tmp_df = detect_patterns(df = tmp_df, patterns = HOOK_PATTERNS)

hook_df = hook_df.fillna(False)

# Brand introduced
hook_df = detect_patterns(df = hook_df, patterns = BRAND_PATTERNS)
# Rename all the brand patterns to end up with _HOOK

# Trick, tip or hack
hook_df = detect_patterns(df = hook_df, patterns = TIPS_PATTERNS, column="text")

# Elipsis
hook_df["ELLIPSIS_HOOK"] = hook_df["text"].apply(lambda x: ellipsis(x))

# Direct questions
hook_df = detect_patterns(df = hook_df, patterns = DIRECT_QUESTION_PATTERNS)
# Rename all the direct questions patterns to end up with _HOOK

# Merge
hook_df = hook_df.merge(tmp_df[["path", "GENERAL_HOOK",	"NEGATIVE_HOOK"]], on="path", how="left")



# 132 -> Answered with the negative hook

81: Text Overlay

82 -83: Creators being used -> Person detection / Associated with a specific category

84: More than one person

85: Creator shown in the first 2 seconds

86 - 88: Scene changes

In [ ]:
video_num_scenes = [scene_num(video) for video in video_paths]
img_num_scenes = [1 for i in len(range(image_paths))]

more_than_1_scene = [1 if video_scenes > 1 else 0 for video_scenes in video_num_scenes]

89: Brand revealed after 6 seconds

In [ ]:
start_second_segment = [raw_audios[audio_keys[i]]["segments"][1]["start"] if
                    len(raw_audios[audio_keys[i]]["segments"]) > 1 else 0
                    for i in range(len(audio_keys))]

text_first_segment = [raw_audios[audio_keys[i]]["segments"][0]["text"] if 
                    len(raw_audios[audio_keys[i]]["segments"]) > 1 else "" 
                    for i in range(len(audio_keys))]

texts = [raw_audios[audio_keys[i]]["text"] for i in range(len(audio_keys))]

# Subtract
text_for_brand_after_6 = [texts[i].replace(text_first_segment[i], "") for i in range(len(texts))]

# If second segment starts after 6 
text_for_brand_after_6 = [text_for_brand_after_6[i] if start_second_segment[i] > 6 else "" for i in range(len(text_for_brand_after_6))]

# Metion the brand
brand_6_sec_df = pd.DataFrame({"path": audio_keys, "text": text_for_brand_after_6})
brand_6_sec_df = detect_patterns(df = brand_6_sec_df, patterns = BRAND_PATTERNS)
# Rename the column

90: Brand being the protagonist

91: Brand placed at top center

92: Opening seconds include speech

In [70]:
end_first_segment = [raw_audios[audio_keys[i]]["segments"][0]["end"] for i in range(len(audio_keys))]

opening_include_speech = [1 if text_first_segment[i] != "" and end_first_segment[i] < 6 else 0
                        for i in range(len(end_first_segment))]

93: In the first 2 seconds do we have more than 10 words. Should be worked with ocr

In [76]:
tmp_lst = [round((len(text_first_segment[i].split(" "))/end_first_segment[i]) * 2) for i in range(len(end_first_segment))]
more_than_10_words = [1 if tmp_lst[i] >= 10 else 0 for i in range(len(tmp_lst))]

94: Energetic music

In [ ]:
df["energetic_music"] = df["bpm_values"].apply(energetic_music)

95: Music in the background

96: Speaking directly to the audience

In [ ]:
df["speaking_directly_to_the_audience"] = df["text"].apply(second_person)

97: Asking questions

In [ ]:
df = detect_patterns(df = df, patterns = DIRECT_QUESTION_PATTERNS)

98: Logo shown tied to the product

99: Emotion sparking in the first 5 seconds -> Emotion Detector

100: Brand associated with the solution

101: Framing

102 - 103: Smiling and recognizable faces

104 - 107: Image quality

108 - 109: Lo-fi / UGC

110 - 121 / 125 - 130: Composition, quality and lighting

122 - 124: Integration

In [ ]:
# Integration after the first 10 seconds
texts_with_brand_start = [[transcription_ts[i][j] if (transcription_ts[i][j][1] == first_brand_appearance_start[i]) and (first_brand_appearance_start[i] != 0) else "" for j in range(len(transcription_ts[i]))] for i in range(len(transcription_ts))]
texts_with_brand_start = [[tpl for tpl in segments if tpl != ''] for segments in texts_with_brand_start]
integration_after_10_secs = [1 if (avg_ts_brand_appearance[i] >= 10) and (avg_ts_brand_appearance[i] > 0) else 0 for i in range(len(avg_ts_brand_appearance))]

# Integration in the first half of the video
int_first_half_video = [1 if (avg_ts_brand_appearance[i] / video_durations[i] <= 0.50) and (avg_ts_brand_appearance[i] > 0) else 0 for i in range(len(avg_ts_brand_appearance))]

# Integration after 10 seconds and in the first half of the video
int_first_half_and_after_10_seconds = [1 if (int_first_half_video[i] and integration_after_10_secs[i]) else 0
    for i in range(len(int_first_half_video))]

In [ ]:
# -> "back to..." or "thanks again to... for sponsoring the video", "click the link in the description box..." 
integration_ending_patterns = INTEGRATION_PATTERNS[IntegrationPattern.INTEGRATION_END]

# Find the last time integration vocabulary was mentioned
int_last = [[(transcription_ts[i][j][1], transcription_ts[i][j][2]) if findall_counter(transcription_ts[i][j][0], integration_ending_patterns) else 0 for j in range(len(transcription_ts[i]))] for i in range(len(transcription_ts))]

# Get the first tupple where the values are not 0
last_end_int_appearance = [next((x for x in reversed(lst_values) if x), 0) for lst_values in int_last]

# Filter to get the last value
end_integration = [x[1] if x != 0 else 0 for x in last_end_int_appearance]

# end_integration - first_brand_appearance_start
integration_duration = [end_integration[i] - first_brand_appearance_start[i] if (end_integration[i] != 0) and (first_brand_appearance_start[i] != 0) and (end_integration[i] > first_brand_appearance_start[i]) else 0 for i in range(len(first_brand_appearance_start))]

# Integration between 90 and 240 seconds
int_between_90_240 = [1 if (integration_duration[i] >= 90) and (integration_duration[i] <= 240) else 0 for i in range(len(integration_duration))]

# # Integration timestamps
# integration_ts = [(first_brand_appearance_start[i], end_integration[i]) if (end_integration[i] != 0) and (first_brand_appearance_start[i] != 0) and (end_integration[i] > first_brand_appearance_start[i]) else 0 for i in range(len(first_brand_appearance_start))]

# # Set as (0, 0) where we have 0
# integration_ts = [(0, 0) if x == 0 else x for x in integration_ts]

131: Key message -> LLM

133 - 140: Conveying the same message

In [78]:
# 134: If call to action _transcription and _ocr true then true?

In [79]:
# 135, 136, 137

140 - 145 -> Single focal point: attention

146 - 149: Main message delivered in the first 6 seconds (Using CTA)

In [85]:
tmp_df = pd.DataFrame({"path": audio_keys, "text": text_first_segment, "end_first_segment": end_first_segment})

# drop columns based on end of the first segment
tmp_df = tmp_df.drop(tmp_df[tmp_df["end_first_segment"] > 3].index)
tmp_df = detect_patterns(df = tmp_df, patterns = CTA_PATTERNS, column="text")

150 - 153: Brand elements at the beginning

In [ ]:
tmp_df = detect_patterns(df = tmp_df, patterns = BRAND_PATTERNS, column="text")

154 - 162 / 165 - 170: Size and dimensions and product visible

171 - 174: Product in usage

175: Product being used during the entire ad

176 - 183: Overlay and Captions (OCR)

184 - 187: Video understandable without sound (dependent)

188: Automated subtitles or closed caption -> Answered with OCR

189 - 207: Sound

In [108]:
# Has voice based on the fact that transcriptions are are only made if we have voice in the audio -> 193 - 196

video_df = pd.DataFrame({"path": video_paths})
video_df["cleaned_path"] = video_df["path"].apply(lambda x: x.split("/")[-1].split(".mp4")[0])
tmp_lst = transcription_df["path"].apply(lambda x: x.split("/")[-1].split(".wav")[0])
video_df["has_voice"] = video_df["cleaned_path"].isin(tmp_lst).astype(int)
video_df["has_voice"]

array([1, 0])

In [ ]:
# Sound included -> 197 - 200
sound_effects = ["Static", "Engine", "Radio"]

def sound_effect(row, col_list):
    total = sum(row[col] for col in col_list)
    return 1 if total > 0.0 else 0

# dynamic_creative_df['sound_effect_detected'] = dynamic_creative_df.apply(sound_effect, col_list=sound_effects, axis=1)


# Sound effect and voice -> 189 - 192


# Voice clearly heard in overlay to music -> 201 - 204


# Include sound -> 205 


# Brand mentioned audible -> 206 

brand_mentioned_audible = [0 if ts_brand[i] == 0 else 1 for i in range(len(ts_brand))]


In [ ]:
# CTA paired with voice -> 207

208 - 211 -> Story arc

212 - 215: Ellipsis

In [12]:
df["has_ellipsis"] = df["text"].apply(lambda x: ellipsis(x))

216 - 221: What's comming next and ask audience opinion to spike curiosity

In [ ]:
transcription_df = detect_patterns(df = transcription_df, patterns = CURIOSITY_PATTERNS)

222: Establishing the problem to bring the product as the solution

223: T&C

224 - 227: 20% of the image occupied by text and text in lowercase. Plus, key message in lowercase

In [ ]:
ocr_df["lowercase_ocr"] = [string.islower() for string in ocr_df["text"]]

228 - 230: Overlay with contrasting visuals and font type sans serif

231 - 232: 5 - 10 words per second

233: Multiple scene changes throught the integration

In [ ]:
# Get the scenes
scenes_videos = [detect(video_paths[i], AdaptiveDetector()) for i in range(len(video_paths))]

# Filter the timestamps
scenes_ts = [[(scenes_videos[i][j][0].get_timecode(), scenes_videos[i][j][1].get_timecode()) for j in range(len(scenes_videos[i]))] for i in range(len(scenes_videos))]

# Get posprocessed scenes
postprocessed_scenes = [[extract_seconds_from_timestamps(scenes_ts[i][j]) for j in range(len(scenes_ts[i]))] for i in range(len(scenes_ts))]

234 - 235: trends

236 - 237: Safe Zone

Model

In [5]:
model = XGBoostService(xgboost_adapter=XGBoostAdapter)
model